# **sys.dm\_db\_index\_physical\_stats**

## O problema da fragmentação de Indices

<span style="color: rgb(22, 22, 22); font-family: &quot;Segoe UI&quot;, SegoeUI, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;"><b style="background-color: yellow;">A fragmentação de índice ocorre quando a ordenação lógica nas páginas de índice não corresponde à ordenação física</b><span style="background-color: rgb(255, 255, 255);">. As páginas podem estar desordenadas durante instruções de modificação de dados de rotina, como&nbsp;</span></span>   `UPDATE`<span style="color:rgb(22, 22, 22);font-family:&quot;Segoe UI&quot;, SegoeUI, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif;font-size:16px;background-color:rgb(255, 255, 255);">,&nbsp;</span>   `DELETE`   <span style="color:rgb(22, 22, 22);font-family:&quot;Segoe UI&quot;, SegoeUI, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif;font-size:16px;background-color:rgb(255, 255, 255);">&nbsp;e&nbsp;</span>   `INSERT`<span style="color:rgb(22, 22, 22);font-family:&quot;Segoe UI&quot;, SegoeUI, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif;font-size:16px;background-color:rgb(255, 255, 255);">. </span> <span style="color: rgb(22, 22, 22); font-family: &quot;Segoe UI&quot;, SegoeUI, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: yellow;"><b>A fragmentação pode apresentar problemas de desempenho devido à E/S adicional necessária para localizar os dados que estão sendo referenciados pelos ponteiros dentro das páginas de índice.</b></span>

<span style="font-size:14px;">Como resolver ?&nbsp;</span>  **Recompilar e reorganizar**

<span style="color:rgb(22, 22, 22);font-family:&quot;Segoe UI&quot;, SegoeUI, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif;font-size:16px;background-color:rgb(255, 255, 255);"><b>A reorganização</b> de um índice é uma operação online que desfragmenta o nível folha do índice (clusterizado e não clusterizado). Esse processo de desfragmentação reordenará fisicamente as páginas de nível folha para corresponder à ordem lógica dos nós da esquerda para a direita. Durante esse processo, as páginas de índice também são compactadas com base no valor fillfactor configurado.</span>

<span style="color:rgb(22, 22, 22);font-family:&quot;Segoe UI&quot;, SegoeUI, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif;font-size:16px;background-color:rgb(255, 255, 255);"><b>Uma recompilação</b> pode estar online ou offline, dependendo do comando executado ou da edição do SQL Server que está sendo usada. Um processo de recompilação offline removerá e recriará o índice. Se você puder fazer isso online, um novo índice será criado em paralelo ao índice existente. Depois de criar o índice, o índice existente será descartado e o novo será renomeado para corresponder ao nome do antigo</span>

## **Estrutura da DMV sys.dm\_db\_index\_physical\_stats**

##   

sys.dm\_db\_index\_physical\_stats (

    { database\_id | NULL | 0 | DEFAULT },

    { object\_id | NULL | 0 | DEFAULT },

    { index\_id | NULL | 0 | -1 | DEFAULT },

    { partition\_number | NULL | 0 | -1 | DEFAULT },

    { mode | 'LIMITED' | 'SAMPLED' | 'DETAILED' }

)

### Parâmetros

1. **database\_id**: ID do banco de dados. Use `NULL` ou `DEFAULT` para o banco de dados atual.
2. **object\_id**: ID da tabela ou visualização indexada. Use `NULL` para todos os objetos.
3. **index\_id**: ID do índice. Use `NULL` para todos os índices.
4. **partition\_number**: Número da partição. Use `NULL` para todas as partições.
5. **mode**: Nível de detalhamento da análise. Pode ser:
    - `'LIMITED'`: Menos detalhado, mais rápido. Analisa apenas o nível raiz e o nível folha do índice.
    - `'SAMPLED'`: Mostra uma amostra de páginas no índice, o que pode ser mais rápido do que uma verificação completa.
    - `'DETAILED'`: Analisa todas as páginas em um índice. É mais detalhado, mas pode ser lento para índices grandes.

## Colunas retornadas

### Colunas Retornadas

A DMV `sys.dm_db_index_physical_stats` retorna várias colunas úteis, incluindo:

- ## Tabela retornada
    
    Expandir a tabela
    
    | Nome da coluna | Tipo de dados | Descrição |
    | --- | --- | --- |
    | database\_id | smallint | Identificação do banco de dados da tabela ou exibição.<br><br>No Banco de Dados SQL do Azure, os valores são exclusivos em um único banco de dados ou em um pool elástico, mas não em um servidor lógico. |
    | object\_id | int | Identificação de objeto da tabela ou exibição na qual o índice se encontra. |
    | index\_id | int | Identificação de um índice.<br><br>0 = Heap. |
    | partition\_number | int | Número de partição de base 1 no objeto proprietário; uma tabela, exibição ou índice.<br><br>1 = Índice ou heap não particionado. |
    | index\_type\_desc | nvarchar(60) | Descrição do tipo de índice:<br><br>– HEAP<br>\- ÍNDICE AGRUPADO<br>\- ÍNDICE NÃO CLUSTERIZADO<br>\- ÍNDICE XML PRIMÁRIO<br>\- ÍNDICE ESTENDIDO<br>\- ÍNDICE XML<br>\- ÍNDICE DE MAPEAMENTO COLUMNSTORE (interno)<br>\- ÍNDICE COLUMNSTORE DELETEBUFFER (interno)<br>\- COLUMNSTORE DELETEÍNDICE DE BITMAP (interno) |
    | hobt\_id | bigint | Heap ou ID de árvore B do índice ou partição.<br><br>Para índices columnstore, essa é a ID de um conjunto de linhas que rastreia dados internos de columnstore para uma partição. Os conjuntos de linhas são armazenados como heaps de dados ou árvores B. Eles têm a mesma ID de índice que o índice columnstore pai. Para obter mais informações, consulte [sys.internal\_partitions (Transact-SQL).](https://learn.microsoft.com/pt-br/sql/relational-databases/system-catalog-views/sys-internal-partitions-transact-sql?view=sql-server-ver16) |
    | alloc\_unit\_type\_desc | nvarchar(60) | Descrição do tipo de unidade de alocação:<br><br>\- IN\_ROW\_DATA<br>\- LOB\_DATA<br>\- ROW\_OVERFLOW\_DATA<br><br>A `LOB_DATA` unidade de alocação contém os dados armazenados em colunas do tipo text, ntext, image, varchar(max), nvarchar(max), varbinary(max) e xml. Para obter mais informações, consulte [Tipos de dados (Transact-SQL)](https://learn.microsoft.com/pt-br/sql/t-sql/data-types/data-types-transact-sql?view=sql-server-ver16).<br><br>A `ROW_OVERFLOW_DATA` unidade de alocação contém os dados armazenados em colunas do tipo varchar(n), nvarchar(n), varbinary(n) e sql\_variant que foram enviadas para fora da linha. |
    | index\_depth | tinyint | Número de níveis de índice.<br><br>1 = Heap, ou `LOB_DATA` ou `ROW_OVERFLOW_DATA` unidade de alocação. |
    | index\_level | tinyint | Nível atual do índice.<br><br>0 para níveis leaf de índice, heaps e `LOB_DATA` /ou `ROW_OVERFLOW_DATA` unidades de alocação.<br><br>Maior que 0 para níveis de índice nonleaf.   index\_level é o mais alto no nível raiz de um índice.<br><br>Os níveis não folha de índices são processados apenas quando _mode_ = DETAILED. |
    | avg\_fragmentation\_in\_percent | float | Fragmentação lógica para índices ou fragmentação de extensão para heaps na `IN_ROW_DATA` unidade de alocação.<br><br>O valor é medido como uma porcentagem e leva em consideração vários arquivos. Para definições de fragmentação lógica e de extensão, consulte Comentários.<br><br>0 para `LOB_DATA` e `ROW_OVERFLOW_DATA` unidades de alocação.<br><br>NULL para heaps quando _mode_ = SAMPLED. |
    | fragment\_count | bigint | Número de fragmentos no nível folha de uma `IN_ROW_DATA` unidade de alocação. Para obter mais informações sobre fragmentos, consulte Comentários.<br><br>NULL para níveis não folha de um índice e `LOB_DATA` /ou `ROW_OVERFLOW_DATA` unidades de alocação.<br><br>NULL para heaps quando _mode_ = SAMPLED. |
    | avg\_fragment\_size\_in\_pages | float | Número médio de páginas em um fragmento no nível folha de uma `IN_ROW_DATA` unidade de alocação.<br><br>NULL para níveis não folha de um índice e `LOB_DATA` /ou `ROW_OVERFLOW_DATA` unidades de alocação.<br><br>NULL para heaps quando _mode_ = SAMPLED. |
    | page\_count | bigint | Número total de páginas de índice ou dados.<br><br>Para um índice, o número total de páginas de índice no nível atual da árvore B na `IN_ROW_DATA` unidade de alocação.<br><br>Para um heap, o número total de páginas de dados na `IN_ROW_DATA` unidade de alocação.<br><br>Para `LOB_DATA` unidades de alocação ou `ROW_OVERFLOW_DATA` , número total de páginas na unidade de alocação. |
    | avg\_page\_space\_used\_in\_percent | float | Porcentagem média de espaço de armazenamento de dados disponível usada em todas as páginas.<br><br>Para um índice, a média se aplica ao nível atual da árvore B na `IN_ROW_DATA` unidade de alocação.<br><br>Para um heap, a média de todas as páginas de dados na `IN_ROW_DATA` unidade de alocação.<br><br>Para `LOB_DATA` unidades de alocação ou `ROW_OVERFLOW_DATA` , a média de todas as páginas na unidade de alocação.<br><br>NULL quando _mode_ = LIMITED. |
    | record\_count | bigint | Número total de registros.<br><br>Para um índice, o número total de registros se aplica ao nível atual da árvore B na `IN_ROW_DATA` unidade de alocação.<br><br>Para um heap, o número total de registros na `IN_ROW_DATA` unidade de alocação.<br><br>Observação: Para um heap, o número de registros retornados dessa função pode não corresponder ao número de linhas retornadas pela execução de um `SELECT COUNT(*)` heap. Isso porque uma linha pode conter vários registros. Por exemplo, em algumas situações de atualização, uma única linha de heap pode ter um registro de encaminhamento e um registro encaminhado como resultado de uma operação de atualização. Além disso, a maioria das linhas LOB grandes é dividida em vários registros no `LOB_DATA` armazenamento.<br><br>Para `LOB_DATA` unidades de alocação ou `ROW_OVERFLOW_DATA` , o número total de registros na unidade de alocação completa.<br><br>NULL quando _mode_ = LIMITED. |
    | ghost\_record\_count | bigint | Número de registros fantasmas prontos para remoção pela tarefa de limpeza fantasma na unidade de alocação.<br><br>0 para níveis não folha de um índice na `IN_ROW_DATA` unidade de alocação.<br><br>NULL quando _mode_ = LIMITED. |
    | version\_ghost\_record\_count | bigint | Número de registros fantasmas retidos por uma transação de isolamento de instantâneo pendente em uma unidade de alocação.<br><br>0 para níveis não folha de um índice na `IN_ROW_DATA` unidade de alocação.<br><br>NULL quando _mode_ = LIMITED. |
    | min\_record\_size\_in\_bytes | int | Tamanho de registro mínimo em bytes.<br><br>Para um índice, o tamanho mínimo do registro se aplica ao nível atual da árvore B na `IN_ROW_DATA` unidade de alocação.<br><br>Para um heap, o tamanho mínimo do `IN_ROW_DATA` registro na unidade de alocação.<br><br>Para `LOB_DATA` unidades de alocação ou `ROW_OVERFLOW_DATA` , o tamanho mínimo do registro na unidade de alocação completa.<br><br>NULL quando _mode_ = LIMITED. |
    | max\_record\_size\_in\_bytes | int | Tamanho de registro máximo em bytes.<br><br>Para um índice, o tamanho máximo do registro se aplica ao nível atual da árvore B na `IN_ROW_DATA` unidade de alocação.<br><br>Para um heap, o tamanho máximo do `IN_ROW_DATA` registro na unidade de alocação.<br><br>Para `LOB_DATA` unidades de alocação ou `ROW_OVERFLOW_DATA` , o tamanho máximo do registro na unidade de alocação completa.<br><br>NULL quando _mode_ = LIMITED. |
    | avg\_record\_size\_in\_bytes | float | Tamanho de registro médio em bytes.<br><br>Para um índice, o tamanho médio do registro se aplica ao nível atual da árvore B na `IN_ROW_DATA` unidade de alocação.<br><br>Para um heap, o tamanho médio do `IN_ROW_DATA` registro na unidade de alocação.<br><br>Para `LOB_DATA` unidades de alocação ou `ROW_OVERFLOW_DATA` , o tamanho médio do registro na unidade de alocação completa.<br><br>NULL quando _mode_ = LIMITED. |
    | forwarded\_record\_count | bigint | Número de registros em um heap com ponteiros encaminhados a outro local de dados. (Esse estado ocorre durante uma atualização, quando não há espaço suficiente para armazenar a nova linha no local original.)<br><br>NULL para qualquer unidade de alocação diferente das `IN_ROW_DATA` unidades de alocação de um heap.<br><br>NULL para heaps quando _mode_ = LIMITED. |
    | compressed\_page\_count | bigint | O número total de páginas compactadas.<br><br>Para heaps, as páginas recém-alocadas não são compactadas PAGE. Um heap é compactado com PAGE em duas condições especiais: quando os dados são importados em massa ou quando um heap é reconstruído. As operações DML típicas que causam alocações de página não são compactadas PAGE. Recompile um heap quando o `compressed_page_count` valor crescer maior do que o limite desejado.<br><br>Para tabelas que têm um índice clusterizado, o `compressed_page_count` valor indica a eficácia da compactação PAGE. |
    | columnstore\_delete\_buffer\_state | tinyint | 0 = NOT\_APPLICABLE<br>1 = OPEN<br>2 = DRENAGEM<br>3 = LAVAGEM<br>4 = APOSENTADORIA<br>5 = PRONTO<br><br>Aplica-se a: SQL Server 2016 (13.x) e versões posteriores, Banco de Dados SQL do Azure e Instância Gerenciada de SQL do Azure |
    | columnstore\_delete\_buffer\_state\_desc | nvarchar(60) | NOT VALID – o índice pai não é um índice columnstore.<br><br>OPEN - deleters e scanners usam isso.<br><br>DRENANDO - os deletores estão drenando, mas os scanners ainda o usam.<br><br>FLUSHING - o buffer é fechado e as linhas no buffer estão sendo gravadas no bitmap de exclusão.<br><br>RETIRING - as linhas no buffer de exclusão fechado foram gravadas no bitmap de exclusão, mas o buffer não foi truncado porque os scanners ainda o estão usando. Novos scanners não precisam usar o buffer de desativação porque o buffer aberto é suficiente.<br><br>READY - Este buffer de exclusão está pronto para uso.<br><br>Aplica-se a: SQL Server 2016 (13.x) e versões posteriores, Banco de Dados SQL do Azure e Instância Gerenciada de SQL do Azure |
    | version\_record\_count | bigint | Essa é a contagem dos registros de versão de linha que estão sendo mantidos nesse índice. Essas versões de linha são mantidas pelo recurso Recuperação [Acelerada de](https://learn.microsoft.com/pt-br/sql/relational-databases/accelerated-database-recovery-concepts?view=sql-server-ver16) Banco de Dados.<br><br>Aplica-se a: SQL Server 2019 (15.x) e versões posteriores e Banco de Dados SQL do Azure |
    | inrow\_version\_record\_count | bigint | Contagem de registros de versão ADR mantidos na linha de dados para recuperação rápida.<br><br>Aplica-se a: SQL Server 2019 (15.x) e versões posteriores e Banco de Dados SQL do Azure |
    | inrow\_diff\_version\_record\_count | bigint | Contagem de registros de versão ADR mantidos na forma de diferenças em relação à versão base.<br><br>Aplica-se a: SQL Server 2019 (15.x) e versões posteriores e Banco de Dados SQL do Azure |
    | total\_inrow\_version\_payload\_size\_in\_bytes | bigint | Tamanho total em bytes dos registros de versão em linha para esse índice.<br><br>Aplica-se a: SQL Server 2019 (15.x) e versões posteriores e Banco de Dados SQL do Azure |
    | offrow\_regular\_version\_record\_count | bigint | Contagem de registros de versão mantidos fora da linha de dados original.<br><br>Aplica-se a: SQL Server 2019 (15.x) e versões posteriores e Banco de Dados SQL do Azure |
    | offrow\_long\_term\_version\_record\_count | bigint | Contagem de registros de versão considerados de longo prazo.<br><br>Aplica-se a: SQL Server 2019 (15.x) e versões posteriores e Banco de Dados SQL do Azur |

# \## Vamos detalhar esses valores

#### `object_id`

- **Descrição**: Identificação do objeto (tabela ou exibição) no qual o índice está presente.
- **Tipo de dados**: `int`
- **Notas**: Cada tabela ou exibição em um banco de dados SQL tem um identificador único, o `object_id`. Este ID é usado para identificar o objeto específico ao qual o índice pertence.

#### `index_id`

- **Descrição**: Identificação de um índice.
- **Tipo de dados**: `int`
- **Notas**: Valores comuns incluem:
    - `0` para heaps (tabelas sem índice clusterizado).
    - `1` para índices clusterizados.
    - Outros valores são para índices não clusterizados.

#### `partition_number`

- **Descrição**: Número de partição (base 1) no objeto proprietário (tabela, exibição ou índice).
- **Tipo de dados**: `int`
- **Notas**: Em tabelas ou índices particionados, este número indica a partição específica. Para tabelas ou índices não particionados, o valor é `1`.

#### `index_type_desc`

- **Descrição**: Descrição do tipo de índice.
- **Tipo de dados**: `nvarchar(60)`
- **Notas**: Valores possíveis incluem:
    - `HEAP`
    - `CLUSTERED INDEX`
    - `NONCLUSTERED INDEX`
    - `XML PRIMARY INDEX`
    - `XML INDEX`
    - Outros tipos de índices específicos.
        
        #### `alloc_unit_type_desc`
        
        #### 
        
        - **Descrição**: Descrição do tipo de unidade de alocação.
        - **Tipo de dados**: `nvarchar(60)`
        - **Notas**: Valores incluem:
            - `IN_ROW_DATA`: Dados armazenados dentro da linha.
            - `LOB_DATA`: Dados de objetos grandes, como `text`, `ntext`, `image`, `varchar(max)`, `nvarchar(max)`, `varbinary(max)`, e `xml`.
            - `ROW_OVERFLOW_DATA`: Dados de colunas `varchar`, `nvarchar`, `varbinary` que excedem o limite de tamanho da linha.

### Exemplos de Uso

1. **Análise Rápida de Todos os Índices em uma Tabela Específica:**

In [ ]:
SELECT *
FROM sys.dm_db_index_physical_stats (
    DB_ID(),               -- database_id
    OBJECT_ID('MinhaTabela'), -- object_id
    NULL,                  -- index_id
    NULL,                  -- partition_number
    'LIMITED'              -- mode
);


> **2.Análise Detalhada de um Índice Específico:**

In [ ]:
SELECT *
FROM sys.dm_db_index_physical_stats (
    DB_ID(),               -- database_id
    OBJECT_ID('MinhaTabela'), -- object_id
    1,                     -- index_id
    NULL,                  -- partition_number
    'DETAILED'             -- mode
);


> **3.Análise Amostrada de Todos os Índices em Todos os Objetos do Banco de Dados Atual:**

In [ ]:
SELECT *
FROM sys.dm_db_index_physical_stats (
    NULL,                  -- database_id
    NULL,                  -- object_id
    NULL,                  -- index_id
    NULL,                  -- partition_number
    'SAMPLED'              -- mode
);
